# Utils

Notebook containing all necessary functions for the conducted analyses

### Imports

In [ ]:
! pip install qmcpy --quiet
! pip install pytictoc --quiet
! pip install POT --quiet

In [ ]:
import numpy as np
import scipy.spatial.distance as distance # distance used for kernel
import qmcpy # QMC points
from pytictoc import TicToc # timer
import ot # Wasserstein distance and Sinkhorn divergence

## Box-Muller transformation

Box-Muller transformation:

In [ ]:
def boxmuller(unif1,unif2):
  u1 = np.sqrt(-2*np.log(unif1))*np.cos(2*np.pi*unif2)
  u2 = np.sqrt(-2*np.log(unif1))*np.sin(2*np.pi*unif2)
  return np.transpose(np.vstack([u1,u2]))

Function generating standard normals using the box-muller transformation:

In [ ]:
def normals(n, d, unif, sv=False):

    # avoid origin
    unif[unif==0] = np.nextafter(0, 1)

    # if d is odd, add one dimension
    if d % 2 != 0:
      dim = d + 1
    else:
      dim = d

    # expand dimensions for SV model
    if sv == True:
      dim = 2+2*d

    # create standard normal samples
    u = np.zeros((n,dim))
    for i in np.arange(0,dim,2):
      u[:,i:(i+2)] = boxmuller(unif[:,i],unif[:,(i+1)])

    # if d is odd, drop one dimension
    if d % 2 != 0 or sv == True:
      u = np.delete(u,-1,1)

    return u

## Kernel

Gaussian kernel $k(x,y)$, its gradient w.r.t. first element $\nabla_1k(x,y)$ and its second derivative w.r.t. to the second and first argument $\nabla_2\nabla_1k(x,y)$:

In [ ]:
def k(x,y,l): 

    x = x.astype('float32')
    y = y.astype('float32')
    
    # dimensions
    d = x.shape[1]
    dims = np.arange(d)
    
    # kernel
    kernel = np.exp(-(1/(2*l**2))*distance.cdist(x,y,'sqeuclidean'))
    
    # first derivative
    grad_1 = -1*np.squeeze(np.subtract.outer(x,y)[:,[dims],:,[dims]], axis=0)*(1/l**2)*np.expand_dims(kernel, axis=0)
    
    #second derivative
    grad_21 = (1/l**2)*(np.expand_dims(np.expand_dims(np.eye(d), axis = 2), axis = 3)-np.einsum('ijk,ljk->iljk',np.squeeze(np.subtract.outer(x,y)[:,[dims],:,[dims]], axis=0),np.squeeze(np.subtract.outer(x,y)[:,[dims],:,[dims]], axis=0))*(1/l**2))*kernel
    
    return list([kernel, grad_1, grad_21])

## MMD$^2$

MMD$^2$ approximation:

In [ ]:
def MMD_approx(n,m,kxx,kxy,kyy):
    
    # first sum
    np.fill_diagonal(kxx, np.repeat(0,n))
    sum1 = np.sum(kxx)
    
    # second sum
    sum2 = np.sum(kxy)
    
    # third sum
    np.fill_diagonal(kyy, np.repeat(0,m))
    sum3 = np.sum(kyy)
    
    return (1/(n*(n-1)))*sum1-(2/(m*n))*sum2+(1/(m*(m-1)))*sum3

MMD$^2$ gradient $\hat{J}$:

In [ ]:
def grad_MMD(p,n,m,grad_g,k1xx,k1xy):
    
    # first sum
    prod1 = np.squeeze(np.einsum('ilj,imjk->lmjk', grad_g, np.expand_dims(k1xx,axis=1)))
    if prod1.ndim==2:
      np.fill_diagonal(prod1[:,:], 0)
      sum1 = np.sum(prod1)
    else:
      for i in range(p):
        np.fill_diagonal(prod1[i,:,:], 0)
      sum1 = np.einsum('ijk->i',prod1)
    
    # second sum
    prod2 = np.squeeze(np.einsum('ilj,imjk->lmjk', grad_g, np.expand_dims(k1xy,axis=1)))
    if prod2.ndim==2:
      sum2 = np.sum(prod2)
    else:
      sum2 = np.einsum('ijk->i',prod2)
    
    return (2/(n*(n-1)))*sum1-(2/(n*m))*sum2

## Information metric

Approximation of the informatin metric $g_U(\theta)$:

In [ ]:
def g_approx(p,n,grad_g,k21xx):
    
    # sum
    grad_g_T = np.einsum('ijk -> jik',grad_g)
    prod1 = np.einsum('ijk, jlkm -> ilkm', grad_g_T, k21xx)
    prod2 = np.einsum('ijkl,jmk->imkl', prod1, grad_g)
    for i in range(p):
        np.fill_diagonal(prod2[i,i,:,:], 0)
    gsum = np.einsum('ijkl->ij', prod2)
    
    return 1/(n*(n-1))*gsum

## Generators

Generator $G_\theta(u)$ and generator gradient $\nabla_\theta G_\theta(u)$ for the **Gaussian location model**:

In [ ]:
# generator
def gen_gaussian(n, d, unif, theta, sigma):

  unif[unif==0] = np.nextafter(0, 1)

  # if d is odd, add one dimension
  if d % 2 != 0:
    dim = d + 1
  else:
    dim = d

  # create standard normal samples
  u = np.zeros((n,dim))
  for i in np.arange(0,dim,2):
    u[:,i:(i+2)] = boxmuller(unif[:,i],unif[:,(i+1)])

  # if d is odd, drop one dimension
  if d % 2 != 0:
    u = np.delete(u,-1,1)

  # generate samples
  x = theta + u*sigma

  return x

# gradient of the generator
def grad_gen_gaussian(n, theta):
    return np.broadcast_to(np.expand_dims(np.eye(theta.shape[0]),axis=2),(theta.shape[0],theta.shape[0],n))

Generator $G_\theta(u)$ for the **beta distribution**:

In [ ]:
# generator
def gen_beta(unif, theta):
  unif[unif==0] = np.nextafter(0, 1)
  alpha = theta[0]
  beta = theta[1]
  return np.expand_dims(np.sum(np.log(unif[:,0:(alpha)]),axis=1)/np.sum(np.log(unif[:,0:(alpha+beta)]),axis=1),axis=1)

Generator $G_\theta(u)$ and generator gradient $\nabla_\theta G_\theta(u)$ for the **g-and-k distribution**:

In [ ]:
# generator
def gen_gandk(z, theta):
    a = theta[0]
    b = theta[1]
    g = theta[2]
    k = np.exp(theta[3])
    g = a+b*(1+0.8*((1-np.exp(-g*z))/(1+np.exp(-g*z))))*((1+z**2)**(k))*z
    return g

# gradient of the generator
def grad_gen_gandk(z,theta):
    a = theta[0]
    b = theta[1]
    g = theta[2]
    k = np.exp(theta[3])
    grad1 = np.ones(z.shape[0])
    grad2 = (1+(4/5)*((1-np.exp(-g*z))/(1+np.exp(-g*z))))*(np.exp(k*np.log(1+z**2)))*z
    grad3 = (8/5)*theta[1]*((np.exp(g*z))/(1+np.exp(g*z))**2)*(np.exp(k*np.log(1+z**2)))*z**2
    grad4 = b*(1+0.8*((1-np.exp(-g*z))/(1+np.exp(-g*z))))*(np.exp(k*np.log(1+z**2)))*np.log(1+z**2)*z
    return np.expand_dims(np.einsum('ij->ji',np.c_[grad1,grad2,grad3,grad4]), axis=0)

Generator $G_\theta(u)$ and generator gradient $\nabla_\theta G_\theta(u)$ for the **stochastic volatility model**:

In [ ]:
# generator
def gen_sv(u, theta):

    t = int((u.shape[0]-1)/2)

    # retrieve parameters
    phi = (np.exp(theta[0])-1)/(np.exp(theta[0])+1)
    kappa = np.exp(theta[1])
    sigma = np.exp(theta[2]/2)
    
    # noise terms
    e = u[0:t,:]
    h1 = u[t,:]*np.sqrt(sigma**2/(1-phi**2))
    eta = u[t+1:,:]*sigma

    # t=1
    h = h1
    y = e[0,:]*kappa*np.exp(0.5*h[0])

    # t=2,...,T
    for i in range(t-1):
      h = np.vstack([h,phi*h[i]+eta[i+1,:]])
      y = np.vstack([y,e[i+1,:]*kappa*np.exp(0.5*h[i+1])])

    return list([y.T,h.T]) # dimensions: nxT and nxT

# generator
def grad_gen_sv(u,h,y,theta):

  # reparameterisation
  phi = (np.exp(theta[0])-1)/(np.exp(theta[0])+1)
  sigma = np.exp(theta[2]/2)

  n = y.shape[0]
  t = y.shape[1]
  eta = (u[t+1:,:]*np.exp(theta[2]/2)).T

  # theta 1
  grad_phi_theta1 = (2*(np.exp(theta[0])))/((np.exp(theta[0])+1)**2)
  grad_h_theta1 = ((np.exp(theta[0]/2)-np.exp(-theta[0]/2))/(np.exp(theta[0]/2))+np.exp(-theta[0]/2))*(h[:,0]/2)
  for i in range(t-1):
    grad_h_theta1 = np.vstack([grad_h_theta1,grad_phi_theta1*h[:,i]+phi*grad_h_theta1[i]])
  grad_y_theta1 = y*grad_h_theta1.T/2
  grad_y_theta1 = np.expand_dims(grad_y_theta1,axis=2)
  
  # theta 2
  grad_y_theta2 = y
  grad_y_theta2 = np.expand_dims(grad_y_theta2,axis=2)

  # theta 3
  grad_eta_theta3 = eta/2
  grad_h_theta3 = h[0,:]/2
  for i in range(n-1):
    grad_h_theta3 = np.vstack([grad_h_theta3,phi*grad_h_theta3[i]+grad_eta_theta3[i+1,:]])
  grad_y_theta3 = y*grad_h_theta3/2
  grad_y_theta3 = np.expand_dims(grad_y_theta3,axis=2)

  # stack together
  grad_stack = np.concatenate((grad_y_theta1,grad_y_theta2,grad_y_theta3), axis=2)

  return np.einsum('ijk->jki',grad_stack) # dimensions: Txpxn

## Sampling

Function to sample from **Gaussian location model**:



In [ ]:
def sample_gaussian(method_sampling,n,d,s,theta):

  ' caveat:                                                                 '
  ' the qmc or qmc_1 sequence have to be fixed before using this function   '

  # odd number of parameters
  if d % 2 != 0: 
    if method_sampling == 'MC':
      unif = np.random.rand(n,d+1)
    if method_sampling == 'QMC':
      unif = qmc_1.gen_samples(n)
    if method_sampling == 'RQMC':
      unif = qmcpy.Halton(d+1).gen_samples(n)

  # even number of parameters
  else: 
    if method_sampling == 'MC':
      unif = np.random.rand(n,d)
    if method_sampling == 'QMC':
      unif = qmc.gen_samples(n)
    if method_sampling == 'RQMC':
      unif = qmcpy.Halton(d).gen_samples(n)

  # use generator  
  x = gen_gaussian(n,d,unif,theta,s)

  return x

Function to sample $R$-times for MC and RQMC from **Gaussian location model**:

In [ ]:
def sample_gaussian_r(n,num,d,s,theta):

  ' caveat:                                                                 '
  ' the qmc or qmc_1 sequence have to be fixed before using this function   '

  # MC
  x_mc = np.zeros((num,np.max(n),d))
  for r in range(num):
    # odd number of parameters
    if d % 2 != 0: 
      unif_mc = np.random.rand(np.max(n),d+1)
    # even number of parameters
    else: 
      unif_mc = np.random.rand(np.max(n),d)
    x = gen_gaussian(np.max(n),d,unif_mc,theta,s)
    x_mc[r,:,:] = x

  # QMC
  # odd number of parameters
  if d % 2 != 0: 
    unif_qmc = qmc_1.gen_samples(np.max(n))
  # even number of parameters
  else: 
    unif_qmc = qmc.gen_samples(np.max(n))
  x_qmc = gen_gaussian(np.max(n),d,unif_qmc,theta,s)

  # RQMC
  x_rqmc = np.zeros((num,np.max(n),d))
  for r in range(num):
    # odd number of parameters
    if d % 2 != 0: 
      unif_rqmc = qmcpy.Halton(d+1).gen_samples(np.max(n))
    # even number of parameters
    else: 
      unif_rqmc = qmcpy.Halton(d).gen_samples(np.max(n))
    x = gen_gaussian(np.max(n),d,unif_rqmc,theta,s)
    x_rqmc[r,:,:] = x

  return list([x_mc,x_qmc,x_rqmc])

Function to sample from **beta distribution**:

In [ ]:
def sample_beta(method_sampling,n,d,theta):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '
  ' as either Halton, Sobol or Lattice points                               '

  if method_sampling == 'MC':
    unif = np.random.rand(n,theta[0]+theta[1])
  if method_sampling == 'QMC':
    unif = qmc.gen_samples(np.max(n))
  if method_sampling == 'RQMC':
    unif = qmcpy.Halton(theta[0]+theta[1]).gen_samples(n)  

  # generate samples
  x = gen_beta(unif,theta)  

  return x

Function to sample $R$-times for MC and RQMC from **beta distribution**:

In [ ]:
def sample_beta_r(qmc_method,n,num,d,theta):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '
  ' as either Halton, Sobol or Lattice points                               '

  # generate n data points using MC and RQMC
  x_mc = np.zeros((num,np.max(n),d))
  x_rqmc = np.zeros((num,np.max(n),d))
  for r in range(num):
    unif_mc = np.random.rand(np.max(n),theta[0]+theta[1])
    if qmc_method == 'halton':
      unif_rqmc = qmcpy.Halton(theta[0]+theta[1]).gen_samples(np.max(n))
    if qmc_method == 'sobol':
      unif_rqmc = qmcpy.Sobol(theta[0]+theta[1], graycode=True).gen_samples(np.max(n))
    if qmc_method == 'lattice':
      unif_rqmc = qmcpy.Lattice(theta[0]+theta[1]).gen_samples(np.max(n))
    x_mc[r,:,:] = gen_beta(unif_mc,theta)
    x_rqmc[r,:,:] = gen_beta(unif_rqmc,theta)

  # QMC
  unif_qmc = qmc.gen_samples(np.max(n))
  x_qmc = gen_beta(unif_qmc,theta)

  return list([x_mc,x_qmc,x_rqmc])

Function to sample from **g-and-k distribution**:

In [ ]:
def sample_gandk(method_sampling,n,d,theta):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '

  # generate uniforms
  if method_sampling == 'MC':
    unif = np.random.rand(n,d+1)
  if method_sampling == 'QMC':
    unif = qmc.gen_samples(n)
  if method_sampling == 'RQMC':
    unif = qmcpy.Halton(d+1).gen_samples(n)

  # generate standard normals  
  z = normals(n,d,unif)

  # generate samples from g-and-k distribution
  x = gen_gandk(z,theta)

  return list([x,z])

Function to sample $R$-times for MC and RQMC from **g-and-k distribution**:

In [ ]:
def sample_gandk_r(n,num,d,theta):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '

  #MC
  x_mc = np.zeros((num,np.max(n),d))
  for r in range(num):
    unif_mc = np.random.rand(np.max(n),d+1)
    z = normals(np.max(n), d, unif_mc)
    x_mc[r,:,:] = gen_gandk(z,theta)

  # QMC
  unif_qmc = qmc.gen_samples(np.max(n))
  z = normals(np.max(n), d, unif_qmc)
  x_qmc = gen_gandk(z,theta)

  # RQMC
  x_rqmc = np.zeros((num,np.max(n),d))
  for r in range(num):
    unif_rqmc = qmcpy.Halton(d+1).gen_samples(np.max(n))
    z = normals(np.max(n), d, unif_rqmc)
    x_rqmc[r,:,:] = gen_gandk(z,theta)

  return list([x_mc,x_qmc,x_rqmc])

Function to sample from **SV model**:

In [ ]:
def sample_sv(method_sampling,n,d,theta):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '

  # generate uniforms
  if method_sampling == 'MC':
    unif = np.random.rand(n,2+2*d)
  if method_sampling == 'QMC':
    unif = qmc.gen_samples(n)
  if method_sampling == 'RQMC':
    unif = qmcpy.Halton(2+2*d).gen_samples(n)

  # generate standard normals  
  u = normals(n,d,unif,sv=True)

  # generate samples from g-and-k distribution and latent variables
  x,h = gen_sv(u.T,theta)

  return list([x,h,u.T])

Function to sample $R$-times for MC and RQMC from **SV model**:

In [ ]:
def sample_sv_r(n,num,d,theta):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '

  return list([x_mc,x_qmc,x_rqmc])

## Optimisation loop

In [ ]:
def optim(model,method_sampling,method_gd,eta,max_it,l,n,m,d,p,y,start,s=2):

  ' caveat:                                                              '
  ' the qmc sequence has to be fixed before using this function          '
  
  ' function arguments:                                                  '   
  ' model:           "gaussian" or "gandk" or "sv"                       '
  ' method_sampling: "MC" or "QMC" or "RQMC"                             '
  ' method_sg:       "SGD" or "NSGD"                                     '
  ' eta:             step size                                           '
  ' max_it:          maximum number of iterations                        '
  ' l:               lengthscale of the kernel                           '
  ' n:               number of samples simulated per iteration           '
  ' m:               number of true samples                              '
  ' d:               number of dimensions                                '
  ' y:               true data set                                       '
  ' start:           start values                                        '
  ' s (optional):    standard deviation in Gaussian location model       '

  # median heuristic if l=-1
  if l == -1:
    l = np.sqrt((1/2)*np.median(distance.cdist(y,y,'sqeuclidean')))

  # pre-define noise for information metric
  noise = [0, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]

  # pre-compute the kernel and its derivatives for the true data points
  kyy = k(y,y,l)
  
  # list for squared MMD
  loss = []  

  # start values
  theta = np.expand_dims(start,axis=0) 

  # create timer instance
  t = TicToc()
  t.tic()

  for i in range(max_it):
    
    # simulate data using the current estimate for theta
    if model == 'gaussian':
      x = sample_gaussian(method_sampling,n,d,s,theta[i,:])
    if model == 'gandk':
      x,z = sample_gandk(method_sampling,n,d,theta[i,:])
    if model == 'sv':
      x,h,u = sample_sv(method_sampling,n,d,theta[i,:])
    
    # calculate kernel and the derivatives
    kxx = k(x,x,l)
    kxy = k(x,y,l)
    
    # calculate the gradient of the generator
    if model == 'gaussian':
      grad_g = grad_gen_gaussian(n,theta[i,:])
    if model == 'gandk':
      grad_g = grad_gen_gandk(z, theta[i,:])
    if model == 'sv':
      grad_g = grad_gen_sv(u,h,x,theta[i,:])
    
    # approximate squared MMD gradient
    if p==1:
      J = np.asmatrix(grad_MMD(p,n,m,grad_g,kxx[1],kxy[1]))
    else:
      J = grad_MMD(p,n,m,grad_g,kxx[1],kxy[1])
    
    # approximate information metric
    if method_gd == 'NSGD':
      g = g_approx(p,n,grad_g,kxx[2])
      # add noise if g can't be inverted
      for j in range(9):
        check = True
        try:
          np.linalg.inv(g + np.eye(p)*noise[j])
        except np.linalg.LinAlgError:
          check = False
        if check:
          break
      g = g + np.eye(p)*noise[j]
    
    # update estimate for theta using NSGD or SGD
    if method_gd == 'NSGD':
        theta = np.vstack([theta,theta[i,:]-eta*np.linalg.inv(g)@J]) # NSGD
    else:
        theta = np.vstack([theta,theta[i,:]-eta*J]) # SGD
    
    # calculate current squared MMD approximation
    loss.append(MMD_approx(n,m,kxx[0],kxy[0],kyy[0]))
    
    # print outputs
    if (i+1)%1000 == 0:
        print('iteration:',i+1,'\nloss:     ', round(loss[i],7),'\nestimate: ',theta[i+1,:],'\ngradient: ', J)

    # stop if nan occurs
    if np.isnan(loss[i]):
      break

  print('-------------------------------------------\nfinal loss:       ', round(loss[i],7), '\nfinal estimate:   ', theta[i+1,:],'\ntotal iterations: ',i+1)
  t.toc() 

  np.savetxt(fname=method_sampling+'_theta.csv', delimiter=",", X=theta)
  np.savetxt(fname=method_sampling+'_loss.csv', delimiter=",", X=loss)

  return list([theta, loss])


### MSE

In [ ]:
def mse(max_it,p,theta1,theta2,theta3,theta_star):
  mse1 = np.zeros((max_it-1,p))
  mse2 = np.zeros((max_it-1,p))
  mse3 = np.zeros((max_it-1,p))
  for l in range(p):
    for j in range(max_it-1):
      mse1[j,l] = np.mean(np.asarray((theta1[1:j+2,l]-theta_star[l]))**2)
      mse2[j,l] = np.mean(np.asarray((theta2[1:j+2,l]-theta_star[l]))**2)
      mse3[j,l] = np.mean(np.asarray((theta3[1:j+2,l]-theta_star[l]))**2)
  return list([mse1,mse2,mse3])

## Convergence of MMD$^2$

Function to calculate MMD$^2$ against $n$ for Gaussian location model, g-and-k distribution and SV model:

In [ ]:
def mmd_conv(model,n,num,d,y,l,theta,s=2):

  # median heuristic if l=-1
  if l == -1:
    l = np.sqrt((1/2)*np.median(distance.cdist(y,y,'sqeuclidean')))

  # kernel
  kyy = k(y,y,l)

  # generate samples
  if model=='gaussian':
    x_mc,x_qmc,x_rqmc = sample_gaussian_r(n,num,d,s,theta)
  if model == 'gandk':
    x_mc,x_qmc,x_rqmc = sample_gandk_r(n,num,d,theta)
  if model == 'sv':
    x_mc,x_qmc,x_rqmc = sample_sv_r(n,num,d,theta)

  # calculate squared MMD for a sequence of n
  MMD_mc = []
  MMD_qmc = []
  MMD_rqmc = []
  MMD_min_mc = []
  MMD_max_mc = []
  MMD_min_rqmc = []
  MMD_max_rqmc = []
  for j in n:

    # R repetitions for MC and RQMC
    mc_ave = []
    rqmc_ave = []
    for r in range(num):
      mc_ave.append(MMD_approx(j,m,k(x_mc[r,:j,:],x_mc[r,:j,:],l)[0],k(x_mc[r,:j,:],y,l)[0],kyy[0]))
      rqmc_ave.append(MMD_approx(j,m,k(x_rqmc[r,:j,:],x_rqmc[r,:j,:],l)[0],k(x_rqmc[r,:j,:],y,l)[0],kyy[0])) 
    
    # append min and max values for MC and RQMC
    MMD_min_mc.append(np.min(np.abs(np.array(mc_ave))))
    MMD_max_mc.append(np.max(np.abs(np.array(mc_ave))))
    MMD_min_rqmc.append(np.min(np.abs(np.array(rqmc_ave))))
    MMD_max_rqmc.append(np.max(np.abs(np.array(rqmc_ave))))

    # append value for QMC and mean values for MC and RQMC
    MMD_mc.append(np.mean(np.abs(np.array(mc_ave))))
    MMD_qmc.append(MMD_approx(j,m,k(x_qmc[:j,:],x_qmc[:j,:],l)[0],k(x_qmc[:j,:],y,l)[0],kyy[0]))
    MMD_rqmc.append(np.mean(np.abs(np.array(rqmc_ave))))
    print('sample size: ', j)

  return list([MMD_mc,MMD_qmc,MMD_rqmc,MMD_min_mc,MMD_max_mc,MMD_min_rqmc,MMD_max_rqmc])

Function to calculate MMD$^2$ against $n$ for beta distribution using Halton sequence, Sobol' sequences or rank-1 lattice:

In [ ]:
# helper to find closest prime number for lattice
def find_next_prime(n):
  for p in range(n+1, 2*n):
        for i in range(2, p):
            if p % i == 0:
                break
        else:
            return p
  return None

In [ ]:
 def mmd_conv_beta(qmc_method,n,num,theta,y,l):

  # median heuristic if l=-1
  if l == -1:
    l = np.sqrt((1/2)*np.median(distance.cdist(y,y,'sqeuclidean'))) 

  # kernel
  kyy = k(y,y,l)

  # generate samples
  x_mc,x_qmc,x_rqmc = sample_beta_r(qmc_method,n,num,d,theta)

  # calculate squared MMD for a sequence of n
  MMD_mc = []
  MMD_qmc = []
  MMD_rqmc = []
  MMD_min_mc = []
  MMD_max_mc = []
  MMD_min_rqmc = []
  MMD_max_rqmc = []
  for j in n:

    # R repetitions for MC and RQMC
    mc_ave = []
    rqmc_ave = []
    for r in range(num):
      mc_ave.append(MMD_approx(j,m,k(x_mc[r,:j,:],x_mc[r,:j,:],l)[0],k(x_mc[r,:j,:],y,l)[0],kyy[0]))
      rqmc_ave.append(MMD_approx(j,m,k(x_rqmc[r,:j,:],x_rqmc[r,:j,:],l)[0],k(x_rqmc[r,:j,:],y,l)[0],kyy[0]))

    # append min and max values for MC and RQMC
    MMD_min_mc.append(np.min(np.abs(np.array(mc_ave))))
    MMD_max_mc.append(np.max(np.abs(np.array(mc_ave))))
    MMD_min_rqmc.append(np.min(np.abs(np.array(rqmc_ave))))
    MMD_max_rqmc.append(np.max(np.abs(np.array(rqmc_ave)))) 

    # append value for QMC and mean values for MC and RQMC
    MMD_mc.append(np.mean(np.abs(np.array(mc_ave))))
    MMD_qmc.append(MMD_approx(j,m,k(x_qmc[:j,:],x_qmc[:j,:],l)[0],k(x_qmc[:j,:],y,l)[0],kyy[0]))
    MMD_rqmc.append(np.mean(np.abs(np.array(rqmc_ave))))

    print('sample size: ', j)

  return list([MMD_mc,MMD_qmc,MMD_rqmc,MMD_min_mc,MMD_max_mc,MMD_min_rqmc,MMD_max_rqmc])

### Convergence of 1-Wasserstein distance

Function to calculate 1-Wasserstein distance against $n$ for Gaussian location model, g-and-k distribution and SV model:

In [ ]:
 def W_conv(model,n,m,num,d,y,theta,s=2): 

  # generate samples
  if model=='gaussian':
    x_mc,x_qmc,x_rqmc = sample_gaussian_r(n,num,d,s,theta)
  if model == 'gandk':
    x_mc,x_qmc,x_rqmc = sample_gandk_r(n,num,d,theta)
  if model == 'sv':
    x_mc,x_qmc,x_rqmc = sample_sv_r(n,num,d,theta)

  # calculate Wasserstein distance for a sequence of n
  W_mc = []
  W_qmc = []
  W_rqmc = []
  W_min_mc = []
  W_max_mc = []
  W_min_rqmc = []
  W_max_rqmc = []
  for j in n:

    mc_ave = []
    rqmc_ave = []

    # equal weights
    a = np.ones((j,)) / j 
    b = np.ones((m,)) / m
    
    # MC and RQMC
    for r in range(num):
      M = ot.dist(x_mc[r,:j,:], y, 'euclidean')
      M /= M.max()
      mc_ave.append(ot.emd2(a, b, M))
      M = ot.dist(x_rqmc[r,:j,:], y, 'euclidean')
      M /= M.max()
      rqmc_ave.append(ot.emd2(a, b, M))
    W_mc.append(np.mean(np.abs(np.array(mc_ave))))
    W_rqmc.append(np.mean(np.abs(np.array(rqmc_ave))))
    
    # append min and max values for MC and RQMC
    W_min_mc.append(np.min(np.abs(np.array(mc_ave))))
    W_max_mc.append(np.max(np.abs(np.array(mc_ave))))
    W_min_rqmc.append(np.min(np.abs(np.array(rqmc_ave))))
    W_max_rqmc.append(np.max(np.abs(np.array(rqmc_ave)))) 

    # QMC
    M = ot.dist(x_qmc[:j,:], y, 'euclidean')
    M /= M.max()
    W_qmc.append(np.abs(ot.emd2(a, b, M)))

    print('sample size: ', j)

  return list([W_mc,W_qmc,W_rqmc,W_min_mc,W_max_mc,W_min_rqmc,W_max_rqmc])

### Convergence of Sinkhorn loss

Function to calculate Sinkhorn loss against $n$ for Gaussian location model, g-and-k distribution and SV model:

In [ ]:
def sink_conv(model,n,m,num,d,y,theta,e,s=2):

  # Sinkhorn divergence for y,y
  b = np.ones((m,)) / m
  M = ot.dist(y, y, 'sqeuclidean')
  M /= M.max()
  sinkyy = ot.sinkhorn2(b, b, M, e)

  # generate samples
  if model=='gaussian':
    x_mc,x_qmc,x_rqmc = sample_gaussian_r(n,num,d,s,theta)
  if model == 'gandk':
    x_mc,x_qmc,x_rqmc = sample_gandk_r(n,num,d,theta)
  if model == 'sv':
    x_mc,x_qmc,x_rqmc = sample_sv_r(n,num,d,theta)

  # calculate Sinkhorn distance for a sequence of n
  sinkxy_qmc = []
  sinkxx_qmc = []
  sink_mc = []
  sink_rqmc = []
  sink_min_mc = []
  sink_max_mc = []
  sink_min_rqmc = []
  sink_max_rqmc = []
  for j in n:

    # equal weights
    a = np.ones((j,)) / j  

    #MC
    mc_avexy = []
    mc_avexx = []
    for r in range(num):
      M = ot.dist(x_mc[r,:j,:], y, 'sqeuclidean')
      M /= M.max()
      mc_avexy.append(ot.sinkhorn2(a, b, M, e))
      M = ot.dist(x_mc[r,:j,:], x_mc[r,:j,:], 'sqeuclidean')
      M /= M.max()
      mc_avexx.append(ot.sinkhorn2(a, a, M, e))

    # QMC
    M = ot.dist(x_qmc[:j,:], y, 'sqeuclidean')
    M /= M.max()
    sinkxy_qmc.append(ot.sinkhorn2(a, b, M, e))
    M = ot.dist(x_qmc[:j,:], x_qmc[:j,:], 'sqeuclidean')
    M /= M.max()
    sinkxx_qmc.append(ot.sinkhorn2(a, a, M, e))

    # RQMC
    rqmc_avexy = []
    rqmc_avexx = []
    for r in range(num):
      M = ot.dist(x_rqmc[r,:j,:], y, 'sqeuclidean')
      M /= M.max()
      rqmc_avexy.append(ot.sinkhorn2(a, b, M, e))
      M = ot.dist(x_rqmc[r,:j,:], x_rqmc[r,:j,:], 'sqeuclidean')
      M /= M.max()
      rqmc_avexx.append(ot.sinkhorn2(a, a, M, e))

    # calculate mean Sinkhorn loss for MC and RQMC
    sink_ave_mc = np.abs(2*np.array(mc_avexy)-np.array(mc_avexx)-sinkyy)
    sink_ave_rqmc = np.abs(2*np.array(rqmc_avexy)-np.array(rqmc_avexx)-sinkyy)
    sink_mc.append(np.mean(sink_ave_mc))
    sink_rqmc.append(np.mean(sink_ave_rqmc))

    # calculate min and max values for MC and RQMC
    sink_max_mc.append(np.max(sink_ave_mc))
    sink_min_mc.append(np.min(sink_ave_mc))
    sink_max_rqmc.append(np.max(sink_ave_rqmc))
    sink_min_rqmc.append(np.min(sink_ave_rqmc))

    print('sample size: ', j)

  # calculate Sinkhorn loss for QMC
  sink_qmc = np.abs(2*np.array(sinkxy_qmc)-np.array(sinkxx_qmc)-sinkyy)

  return list([sink_mc,sink_qmc,sink_rqmc,sink_min_mc,sink_max_mc,sink_min_rqmc,sink_max_rqmc])